In [1]:
import pandas as pd

#Notebookに複数のDataFrameを(水平に)出力する
class HorizontalDisplay: 
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        template = '<div style="float: left; padding: 10px;">{0}</div>'
        return "\n".join(template.format(arg._repr_html_())
                         for arg in self.args)

parents =[
    {'p_age':40, 'p_name':'A'},
    {'p_age':30, 'p_name':'B'},
    {'p_age':20, 'p_name':'C'},
]

children = [
    {'ch_age':20, 'code_school': 'S005','id_Parent':1},
    {'ch_age':17, 'code_school': 'S004','id_Parent':1},
    {'ch_age':15, 'code_school': 'S004','id_Parent':1},
    {'ch_age':10, 'code_school': 'S002','id_Parent':2},
    {'ch_age':1, 'code_school': 'S001','id_Parent':2}
]

schools = [
    {'code_school':'S000', 'type':'家'},
    {'code_school':'S001', 'type':'保育園'},
    {'code_school':'S002', 'type':'小学校'},
    {'code_school':'S003', 'type':'中学校'},
    {'code_school':'S004', 'type':'高校'},
    {'code_school':'S005', 'type':'大学'},
    {'code_school':'S006', 'type':'大学院'},
]

df_parent = pd.DataFrame(parents)
df_children = pd.DataFrame(children)
df_schools = pd.DataFrame(schools)

display(HorizontalDisplay(df_parent, df_children, df_schools))

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session, relationship
from sqlalchemy.orm import Session #データ型取得
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, Float, String, DateTime, Boolean, ForeignKey #SQLテーブルのカラム設定用
from sqlalchemy import desc #降順ソート
import pandas as pd
from typing import Optional, List, Dict

engine = create_engine('sqlite:///:memory:', echo=True) #メモリ上にDB作成(一時的な処理用)

Session = scoped_session(
    sessionmaker(
        autocommit=False, #commit自動化の設定
        autoflush=True, #flush自動化の設定
        bind = engine
    )
)

Base = declarative_base() #DB基底クラスの作成
Base.query = Session.query_property() #DBクエリの設定

class Parent(Base):
    __tablename__ = 'parent' #テーブル名の作成
    id = Column(Integer, primary_key=True) #データログid設定：主キー制約
    age = Column(Float)
    name = Column(String)
    
    children = relationship('Child', backref='parent') #Childテーブルとのリレーション設定
    
    def __init__(self, age, name):
        self.age = age
        self.name = name
        

class Child(Base):
    __tablename__ = 'child' #テーブル名の作成
    id = Column(Integer, primary_key=True) #データログid設定：主キー制約
    ch_age = Column(Float)
    code_school = Column(String, ForeignKey('school.code_school'))
    id_Parent = Column(Integer, ForeignKey('parent.id')) #外部キー設定
    
    # parents = relationship('Parent', backref='child') #backrefのため記載不要※back_populatesの場合は要記載
    # schools = relationship('School', backref='child') #backrefのため記載不要※back_populatesの場合は要記載

    def __init__(self, ch_age, code_school, id_Parent):
        self.ch_age = ch_age
        self.code_school = code_school
        self.id_Parent = id_Parent

class School(Base):
    __tablename__ = 'school'
    code_school = Column(String, primary_key=True)
    type = Column(String)
    
    childen = relationship('Child', backref='school') #Parentクラスとの関連付け

    def __init__(self, code_school, type):
        self.code_school = code_school
        self.type = type
    
    
def init_DB():
    Base.metadata.create_all(bind=engine) #DB作成/初期化
    
init_DB()



2023-03-17 03:02:52,558 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 03:02:52,560 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parent")
2023-03-17 03:02:52,560 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 03:02:52,562 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("parent")
2023-03-17 03:02:52,563 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 03:02:52,563 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("child")
2023-03-17 03:02:52,564 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 03:02:52,565 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("child")
2023-03-17 03:02:52,566 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 03:02:52,567 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("school")
2023-03-17 03:02:52,567 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 03:02:52,568 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("school")
2023-03-17 03:02:52,568 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03

C:\Users\tsc95\AppData\Local\Temp\ipykernel_19156\3497977446.py:20: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base() #DB基底クラスの作成


In [5]:

parents[0]

{'p_age': 40, 'p_name': 'A'}

In [3]:
def insert_data():
    for p in parents:
        p_ins = Parent(p['p_age'], p['p_name'])
        Session.add(p_ins)
    for chl in children:
        chl = Child(chl['ch_age'], chl['code_school'], chl['id_Parent'])
        Session.add(chl)
    for school in schools:
        school = School(school['code_school'], school['type'])
        Session.add(school)
    Session.commit()
    
insert_data()

2023-03-17 03:02:55,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 03:02:55,996 INFO sqlalchemy.engine.Engine INSERT INTO parent (age, name) VALUES (?, ?)
2023-03-17 03:02:55,997 INFO sqlalchemy.engine.Engine [generated in 0.00087s] (40.0, 'A')
2023-03-17 03:02:55,998 INFO sqlalchemy.engine.Engine INSERT INTO parent (age, name) VALUES (?, ?)
2023-03-17 03:02:55,998 INFO sqlalchemy.engine.Engine [cached since 0.002311s ago] (30.0, 'B')
2023-03-17 03:02:55,999 INFO sqlalchemy.engine.Engine INSERT INTO parent (age, name) VALUES (?, ?)
2023-03-17 03:02:56,000 INFO sqlalchemy.engine.Engine [cached since 0.003826s ago] (20.0, 'C')
2023-03-17 03:02:56,001 INFO sqlalchemy.engine.Engine INSERT INTO school (code_school, type) VALUES (?, ?)
2023-03-17 03:02:56,002 INFO sqlalchemy.engine.Engine [generated in 0.00064s] (('S000', '家'), ('S001', '保育園'), ('S002', '小学校'), ('S003', '中学校'), ('S004', '高校'), ('S005', '大学'), ('S006', '大学院'))
2023-03-17 03:02:56,003 INFO sqlalchemy.engine.Engi

In [5]:
def showdfs():
    df_P = pd.read_sql_query('SELECT * FROM parent', engine)
    df_Chl = pd.read_sql_query('SELECT * FROM child', engine)
    df_Sch = pd.read_sql_query('SELECT * FROM school', engine)

    display(HorizontalDisplay(df_P, df_Chl, df_Sch))

showdfs()


2023-03-17 23:32:43,804 INFO sqlalchemy.engine.Engine SELECT * FROM parent
2023-03-17 23:32:43,806 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 23:32:43,815 INFO sqlalchemy.engine.Engine SELECT * FROM child
2023-03-17 23:32:43,816 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 23:32:43,819 INFO sqlalchemy.engine.Engine SELECT * FROM school
2023-03-17 23:32:43,820 INFO sqlalchemy.engine.Engine [raw sql] ()
